In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import torch
from cosypose.config import LOCAL_DATA_DIR

friendly_names = (LOCAL_DATA_DIR / 'bop_datasets/ycbv/ycbv_friendly_names.txt').read_text()
friendly_names = {str(int(l.split(' ')[0])): l.split(' ')[1] for l in friendly_names.split('\n')[:-1]}

def make_errors_dict(key, result_id):
    x = torch.load(LOCAL_DATA_DIR / 'results' / result_id / 'results.pth.tar')
    eval_df = x['dfs']['posecnn_init/refiner/iteration=2']
    scores = eval_df[f'{key}_ntop=1_matching=CLASS']['gt']['AUC/objects']
    object_names = scores.objects.values
    this_friendly_names = [friendly_names[str(int(x.split('_')[1]))] for x in object_names]
    this_scores = {this_friendly_names[n]: scores.values[n] for n in range(len(scores))}
    return this_scores


def make_summary_df(result_id):
    metrics = dict()
    metrics_k = ['ADD', 'ADD-S', 'ADD(-S)']
    for k in metrics_k:
        scores = make_errors_dict(k, result_id)
        metrics[k] = scores
    names = list(metrics[metrics_k[0]].keys())
    names += ['mean']
    df = {'object': names}
    for k in metrics_k:
        df[k] = [metrics[k][o] * 100 if o != 'mean' else np.mean(list(metrics[k].values())) * 100 for o in names]
    df = pd.DataFrame(df)
    return pd.DataFrame(df).set_index('object')

result_id = 'ycbv-n_views=1--5154971130'
df = make_summary_df(result_id)

pd.options.display.float_format = '{:,.1f}'.format
df